Enable All Tracing Without Filtering

In [ ]:
var builder = WebApplication.CreateBuilder(args);

builder.Services.AddOpenTelemetry()
    .WithTracing(tracing => tracing.AddAspNetCoreInstrumentation()
                                   .AddHttpClientInstrumentation()
                                   .AddSqlClientInstrumentation() // ❌ Captures every query
                                   .AddOtlpExporter()); // ❌ Sends too much data

var app = builder.Build();
app.MapGet("/", () => "Hello World!");
app.Run();

Use Filters and Sampling

In [ ]:
var builder = WebApplication.CreateBuilder(args);

builder.Services.AddOpenTelemetry()
    .WithTracing(tracing => tracing
        .AddAspNetCoreInstrumentation(options => 
        {
            options.Filter = request => request.Path != "/health"; // ✅ Ignore health checks
        })
        .AddHttpClientInstrumentation()
        .AddSqlClientInstrumentation(options =>
        {
            options.SetDbStatementForText = false; // ✅ Avoid logging full SQL queries
        })
        .SetSampler(new ParentBasedSampler(new TraceIdRatioBasedSampler(0.2))) // ✅ Sample only 20% of traces
        .AddOtlpExporter());

var app = builder.Build();
app.MapGet("/", () => "Hello World!");
app.Run();